Fundamentos de ciência de dados - Luiz Davi e Thiago Cavalcanti

In [3]:
from nba_api_jogadores_utils import obter_logs_dos_jogos_por_jogador
from constantes_jogadores import JOGADOR_1, JOGADOR_2, JOGADOR_3, JOGADOR_1_ID, JOGADOR_2_ID, JOGADOR_3_ID, TEMPORADA_ATUAL, RESULTADOS_PATH, TIME_ID
import pandas as pd
from nba_api_utils import obter_time, salvar_dataset_csv

# RF3 - O sistema deve fornecer os dados das partidas que o usuário escolher. 
Ex. Partidas contra time X e apresentar esses dados conforme Tabela 3.

In [23]:
def retornar_estatisticas_do_jogador(logs, nome_jogador):
    logs['PLAYER'] = nome_jogador
    logs['HOME/AWAY'] = logs['MATCHUP'].apply(lambda x: 'Casa' if 'vs.' in x else 'Fora')
    logs['SCOREBOARD'] = None

    logs['WL'] = logs['WL'].map({'W': 'Vitória', 'L': 'Derrota'})
    logs['MATCHUP'] = logs['MATCHUP'].apply(lambda x: x.split(' ')[-1])
    
    dataset = logs[['PLAYER', 'GAME_DATE', 'MATCHUP', 'WL', 'HOME/AWAY', 'PTS', 'REB', 'AST', 'SCOREBOARD', 'Game_ID', 'FG3A', 'FG3M', 'MIN']]
    dataset.columns = ['Jogador', 'Data do jogo', 'Adversário', 'Vitória ou derrota', 'Casa ou fora', 'Pontos', 'Rebotes', 'Assistências', 'Placar', 'Id do jogo', 'Tentativas de cesta de 3 pontos', 'Cestas de 3 pontos marcadas', 'Tempo de permanência']
    return dataset

In [24]:
from nba_api.stats.endpoints import boxscoretraditionalv2

def retornar_placar_da_partida(eh_visitante, id_do_jogo):
    jogo_boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=id_do_jogo)
    dados_boxscore = jogo_boxscore.get_data_frames()[0]

    if eh_visitante:
        placar_adversario = dados_boxscore[dados_boxscore['TEAM_ID'] == TIME_ID]['PTS'].sum()
        placar_time = dados_boxscore[dados_boxscore['TEAM_ID'] != TIME_ID]['PTS'].sum()
    else:
        placar_time = dados_boxscore[dados_boxscore['TEAM_ID'] == TIME_ID]['PTS'].sum()
        placar_adversario = dados_boxscore[dados_boxscore['TEAM_ID'] != TIME_ID]['PTS'].sum()

    return f'{int(placar_time)} - {int(placar_adversario)}'

In [25]:
import time
def adicionar_placares_no_dataset(estatisticas):
    for index, row in estatisticas.iterrows():
        eh_visitante = row['Casa ou fora']
        id_do_jogo = row['Id do jogo']
        
        placar = retornar_placar_da_partida(eh_visitante, id_do_jogo)
        
        time.sleep(1) # Pode ser removido, caso não esteja acontecendo timeout na requisição para a API
        
        estatisticas.at[index, 'Placar'] = placar

## Diferença para o RF2

In [20]:
adversario =  obter_time("Chicago Bulls")
adversario

id                 1610612741
full_name       Chicago Bulls
abbreviation              CHI
nickname                Bulls
city                  Chicago
state                Illinois
year_founded             1966
Name: 4, dtype: object

In [22]:
def obter_logs_dos_jogos_contra_adversario(jogador_id, temporada, adversario):
    logs = obter_logs_dos_jogos_por_jogador(jogador_id, temporada)
    
    logs_filtrados = logs[logs['MATCHUP'].str.contains(adversario, regex=False)]

    return logs_filtrados

In [26]:
def retornar_estatisticas_do_jogador_contra_adversario(jogador_id, nome_jogador, temporada, adversario):
    logs = obter_logs_dos_jogos_contra_adversario(jogador_id, temporada, adversario)
    
    if logs.empty:
        print(f"Nenhum jogo encontrado contra {adversario} para {nome_jogador}.")
        return pd.DataFrame()

    dataset = retornar_estatisticas_do_jogador(logs, nome_jogador)
    return dataset

## Obtendo estatísticas de cada jogador

In [34]:
adversario_escolhido = adversario['abbreviation']
adversario_escolhido

'CHI'

In [35]:
estatisticas_jogador_1 = retornar_estatisticas_do_jogador_contra_adversario(JOGADOR_1_ID, JOGADOR_1, TEMPORADA_ATUAL, adversario_escolhido)

adicionar_placares_no_dataset(estatisticas_jogador_1)
estatisticas_jogador_1 = estatisticas_jogador_1.drop(columns=['Id do jogo'])

estatisticas_jogador_1

,Jogador,Data do jogo,Adversário,Vitória ou derrota,Casa ou fora,Pontos,Rebotes,Assistências,Placar,Tentativas de cesta de 3 pontos,Cestas de 3 pontos marcadas,Tempo de permanência
1,Nikola Jokić,"JAN 27, 2025",CHI,Derrota,Fora,33,12,14,129 - 121,4,1,40


In [36]:
estatisticas_jogador_2 = retornar_estatisticas_do_jogador_contra_adversario(JOGADOR_2_ID, JOGADOR_2, TEMPORADA_ATUAL, adversario_escolhido)

adicionar_placares_no_dataset(estatisticas_jogador_2)

estatisticas_jogador_2 = estatisticas_jogador_2.drop(columns=['Id do jogo'])

estatisticas_jogador_2

,Jogador,Data do jogo,Adversário,Vitória ou derrota,Casa ou fora,Pontos,Rebotes,Assistências,Placar,Tentativas de cesta de 3 pontos,Cestas de 3 pontos marcadas,Tempo de permanência
1,Michael Porter Jr.,"JAN 27, 2025",CHI,Derrota,Fora,18,7,0,129 - 121,8,2,32


In [37]:
estatisticas_jogador_3 = retornar_estatisticas_do_jogador_contra_adversario(JOGADOR_3_ID, JOGADOR_3, TEMPORADA_ATUAL, adversario_escolhido)

adicionar_placares_no_dataset(estatisticas_jogador_3)
estatisticas_jogador_3 = estatisticas_jogador_3.drop(columns=['Id do jogo'])

estatisticas_jogador_3

,Jogador,Data do jogo,Adversário,Vitória ou derrota,Casa ou fora,Pontos,Rebotes,Assistências,Placar,Tentativas de cesta de 3 pontos,Cestas de 3 pontos marcadas,Tempo de permanência
1,Jamal Murray,"JAN 27, 2025",CHI,Derrota,Fora,18,6,6,129 - 121,5,1,40


In [38]:
estatisticas_jogadores = pd.concat([estatisticas_jogador_1, estatisticas_jogador_2, estatisticas_jogador_3], ignore_index=True)

estatisticas_jogadores['Data do jogo'] = pd.to_datetime(estatisticas_jogadores['Data do jogo'], format='%b %d, %Y')
estatisticas_jogadores = estatisticas_jogadores.sort_values(by='Data do jogo')

estatisticas_jogadores

,Jogador,Data do jogo,Adversário,Vitória ou derrota,Casa ou fora,Pontos,Rebotes,Assistências,Placar,Tentativas de cesta de 3 pontos,Cestas de 3 pontos marcadas,Tempo de permanência
0,Nikola Jokić,2025-01-27,CHI,Derrota,Fora,33,12,14,129 - 121,4,1,40
1,Michael Porter Jr.,2025-01-27,CHI,Derrota,Fora,18,7,0,129 - 121,8,2,32
2,Jamal Murray,2025-01-27,CHI,Derrota,Fora,18,6,6,129 - 121,5,1,40


In [39]:
salvar_dataset_csv(estatisticas_jogadores, f"{RESULTADOS_PATH}/rf3_estatisticas_jogadores_partida.csv")